# Evaluation

## Load

In [27]:
import torch
from sklearn.metrics import top_k_accuracy_score

from kddirkit.metrics import metric
%matplotlib inline

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import os
import sys
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.metrics import average_precision_score
from ast import literal_eval

""
sns.set_context("notebook", font_scale=1.5)
plt.rcParams['figure.figsize'] = (17, 5)

import platform
if 'Windows' in platform.platform():
    ROOT_PATH = "D:/PycharmProjects/HMAN"
else:
    ROOT_PATH = "/home/xkliu/PycharmProjects/HMAN"
RAW_DATA_PATH = ROOT_PATH  + "/raw_data"
DATA_PATH = ROOT_PATH + "/data"
os.chdir(ROOT_PATH)
sys.path.append("./")

from kddirkit.utils import utils


In [28]:
import logging
import torch.nn as nn
import numpy as np

from kddirkit.networks.encoders import SentenceEncoder
from kddirkit.networks.models import BaselineModel
from kddirkit.config import *
# from kddirkit.dataloaders import LoadNYT, LoadHierData
from kddirkit.frameworks import Trainer
from kddirkit.losses.FocalLoss import FocalLoss

project_name = 'HMAN'

logger = logging.getLogger(project_name)

In [29]:
track_dtype = {'track_id': int, 'album_id': int, 'album_type': str, 'artist_id': int, 'set_split': str,
               'set_subset': str, 'track_genre_top': str, 'track_genres': str, 'track_genres_all': str,
               'track_title': str}
genres_converters = {'track_genres': literal_eval, 'track_genres_all': literal_eval}
medium_data = pd.read_csv(RAW_DATA_PATH + '/medium_data.csv', converters=genres_converters)

In [30]:
medium_data_train = pd.read_csv(RAW_DATA_PATH + '/medium_data_train.csv', converters=genres_converters)
medium_data_test = pd.read_csv(RAW_DATA_PATH + '/medium_data_test.csv', converters=genres_converters)
medium_data_val = pd.read_csv(RAW_DATA_PATH + '/medium_data_val.csv', converters=genres_converters)

In [31]:
medium_instance_triple = np.load(DATA_PATH + '/' + 'medium_instance_triple.npy')
medium_instance_scope = np.load(DATA_PATH + '/' + 'medium_instance_scope.npy')
medium_label = np.load(DATA_PATH + '/' + 'medium_label.npy')

In [32]:
medium_train_instance_triple = np.load(DATA_PATH + '/' + 'medium_train_instance_triple.npy')
medium_train_instance_scope = np.load(DATA_PATH + '/' + 'medium_train_instance_scope.npy')
medium_train_label = np.load(DATA_PATH + '/' + 'medium_train_label.npy')

In [33]:
medium_val_instance_triple = np.load(DATA_PATH + '/' + 'medium_val_instance_triple.npy')
medium_val_instance_scope = np.load(DATA_PATH + '/' + 'medium_val_instance_scope.npy')
medium_val_label = np.load(DATA_PATH + '/' + 'medium_val_label.npy')

In [34]:
medium_test_entity_pair = np.load(DATA_PATH + '/' + 'medium_test_entity_pair.npy')
medium_test_entity_scope = np.load(DATA_PATH + '/' + 'medium_test_entity_scope.npy')
medium_test_label = np.load(DATA_PATH + '/' + 'medium_test_label.npy')

In [35]:
medium_label_transform = np.load(DATA_PATH + '/' + 'medium_label_transform.npy')
medium_train_label_transform = np.load(DATA_PATH + '/' + 'medium_train_label_transform.npy')
medium_val_label_transform = np.load(DATA_PATH + '/' + 'medium_val_label_transform.npy')
medium_test_label_transform = np.load(DATA_PATH + '/' + 'medium_test_label_transform.npy')
medium_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_label_bottom_transform.npy')
medium_train_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_train_label_bottom_transform.npy')
medium_val_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_val_label_bottom_transform.npy')
medium_test_label_bottom_transform = np.load(DATA_PATH + '/' + 'medium_test_label_bottom_transform.npy')

In [36]:
col_name = ['track_id', 'album_id', 'album_type', 'artist_id', 'set_split', 'set_subset', 'track_genres_top', 'track_genre', 'track_genres_all']
medium_data_train_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_train_sort.txt', sep ='-----',  skiprows =1, names  = col_name)
medium_data_val_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_VAL_sort.txt', sep = '-----',  skiprows =1, names  = col_name)
medium_data_test_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_test_sort.txt', sep = '-----', skiprows =1, names  = col_name)

C:\Users\Johan\AppData\Local\Temp\ipykernel_32840\691608824.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  medium_data_train_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_train_sort.txt', sep ='-----',  skiprows =1, names  = col_name)
C:\Users\Johan\AppData\Local\Temp\ipykernel_32840\691608824.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  medium_data_val_sort = pd.read_csv(RAW_DATA_PATH + '/' + 'medium_data_VAL_sort.txt', sep = '-----',  skiprows =1, names  = col_name)
C:\Users\Johan\AppData\Local\Temp\ipykernel_32840\691608824.py:4: ParserWarning: Falling back to the 'python' engine 

In [37]:
medium_data_train_sort

,track_id,album_id,album_type,artist_id,set_split,set_subset,track_genres_top,track_genre,track_genres_all
0,2,1,Album,1,training,small,Hip-Hop,[21],[21]
1,5,1,Album,1,training,small,Hip-Hop,[21],[21]
2,3,1,Album,1,training,medium,Hip-Hop,[21],[21]
3,134,1,Album,1,training,medium,Hip-Hop,[21],[21]
4,10666,1,Album,1,training,medium,Hip-Hop,[21],[21]
...,...,...,...,...,...,...,...,...,...
19917,155297,22935,Album,24354,training,medium,Instrumental,"[18, 107, 1235]","[107, 18, 1235]"
19918,155298,22936,Album,22050,training,medium,Folk,"[17, 103]","[17, 103]"
19919,155306,22936,Album,22050,training,medium,Folk,"[17, 103]","[17, 103]"
19920,155307,22937,Live Performance,7820,training,medium,Experimental,[1],"[1, 38]"


In [39]:
# Load metadata and features.
tracks = utils.load(RAW_DATA_PATH + '/fma_metadata/tracks.csv')
genres = utils.load(RAW_DATA_PATH + '/fma_metadata/genres.csv')
features = utils.load(RAW_DATA_PATH + '/fma_metadata/features.csv')
echonest = utils.load(RAW_DATA_PATH + '/fma_metadata/echonest.csv')

In [40]:
medium_data_train_sort.track_id

0             2
1             5
2             3
3           134
4         10666
          ...  
19917    155297
19918    155298
19919    155306
19920    155307
19921    155314
Name: track_id, Length: 19922, dtype: int64

In [41]:
small = tracks['set', 'subset'] <= 'medium'

train = tracks['set', 'split'] == 'training'
small = tracks['set', 'subset'] <= 'medium'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

chroma_feature = features.loc[medium_data_train_sort.track_id, 'chroma_cens']
tonnetz_feature = features.loc[medium_data_train_sort.track_id, 'tonnetz']
mfcc_feature = features.loc[medium_data_train_sort.track_id, 'mfcc']
centroid_feature = features.loc[medium_data_train_sort.track_id, 'spectral_centroid']
bandwidth_feature = features.loc[medium_data_train_sort.track_id, 'spectral_bandwidth']
contrast_feature = features.loc[medium_data_train_sort.track_id, 'spectral_contrast']
rolloff_feature = features.loc[medium_data_train_sort.track_id, 'spectral_rolloff']
rmse_feature = features.loc[medium_data_train_sort.track_id, 'rmse']
zcr_feature = features.loc[medium_data_train_sort.track_id, 'zcr']
X_train = np.array(pd.concat([chroma_feature, tonnetz_feature, mfcc_feature, centroid_feature, bandwidth_feature, contrast_feature, rolloff_feature, rmse_feature, zcr_feature], axis = 1))

chroma_feature = features.loc[medium_data_test_sort.track_id, 'chroma_cens']
tonnetz_feature = features.loc[medium_data_test_sort.track_id, 'tonnetz']
mfcc_feature = features.loc[medium_data_test_sort.track_id, 'mfcc']
centroid_feature = features.loc[medium_data_test_sort.track_id, 'spectral_centroid']
bandwidth_feature = features.loc[medium_data_test_sort.track_id, 'spectral_bandwidth']
contrast_feature = features.loc[medium_data_test_sort.track_id, 'spectral_contrast']
rolloff_feature = features.loc[medium_data_test_sort.track_id, 'spectral_rolloff']
rmse_feature = features.loc[medium_data_test_sort.track_id, 'rmse']
zcr_feature = features.loc[medium_data_test_sort.track_id, 'zcr']
X_test = np.array(pd.concat(
    [chroma_feature, tonnetz_feature, mfcc_feature, centroid_feature, bandwidth_feature,
     contrast_feature, rolloff_feature, rmse_feature, zcr_feature], axis=1))

y_train = tracks.loc[medium_data_train_sort.track_id, ('track', 'genre_top')]
y_val = tracks.loc[medium_data_val_sort.track_id, ('track', 'genre_top')]
y_test = tracks.loc[medium_data_test_sort.track_id, ('track', 'genre_top')]
# X_train = features.loc[medium_data_train_sort.track_id, 'chroma_cens']
X_val= features.loc[medium_data_val_sort.track_id, 'chroma_cens']
# X_test = features.loc[medium_data_test_sort.track_id, 'chroma_cens']

print('{} training examples, {} testing examples'.format(y_train.size, y_test.size))
print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

19922 training examples, 2573 testing examples
350 features, 16 classes


In [42]:
from sklearn.neural_network import MLPClassifier

# Be sure training samples are shuffled.
X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=42)


X_train_np = np.array(X_train).astype('float32')
X_test_np = np.array(X_test).astype('float32')
X_val_np = np.array(X_val).astype('float32')

y_train_np = np.argmax(pd.get_dummies(y_train).to_numpy(), axis=1)
y_test_np = np.argmax(pd.get_dummies(y_test).to_numpy(), axis = 1)
y_val_np = np.argmax(pd.get_dummies(y_val).to_numpy(), axis = 1)
X_train_np

array([[-3.1772271e-01,  4.7041556e-01,  8.3269000e-01, ...,
         0.0000000e+00,  9.0402907e-01,  2.1812849e-02],
       [ 4.4840875e-01,  2.6973519e-01,  7.3415899e-01, ...,
         2.9296875e-03,  6.4523172e+00,  9.2959560e-02],
       [-1.8923731e-01, -4.3042916e-01, -5.8130997e-01, ...,
         0.0000000e+00,  1.1771928e+00,  1.9115636e-02],
       ...,
       [-5.8614880e-02,  4.1765541e-02,  1.6710621e-01, ...,
         0.0000000e+00,  1.2897763e+00,  2.7492732e-02],
       [ 7.5126600e-01,  3.2632694e+00,  2.6064632e+00, ...,
         5.3710938e-03,  1.3809724e+00,  4.3747392e-02],
       [-3.3007073e-01, -7.5551963e-01, -1.0224332e+00, ...,
         9.2773438e-03,  1.2566203e+00,  7.9019656e-03]], dtype=float32)

In [43]:
# Standardize features by removing the mean and scaling to unit variance.
scaler = skl.preprocessing.StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

# Support vector classification.
X_train

array([[-0.30672131,  0.27931084,  0.11356005, ..., -0.51318961,
        -0.81050529, -0.72263236],
       [ 0.18826375,  0.14552122,  0.09464844, ...,  0.07462511,
         1.1971453 ,  2.10486865],
       [-0.22370899, -0.32126444, -0.15783681, ..., -0.51318961,
        -0.71166087, -0.82982455],
       ...,
       [-0.13931596, -0.00646161, -0.01418914, ..., -0.51318961,
        -0.67092247, -0.49690338],
       [ 0.38393488,  2.14125101,  0.45401017, ...,  0.56447071,
        -0.63792314,  0.14908666],
       [-0.31469917, -0.53799579, -0.24250403, ...,  1.34822367,
        -0.68292   , -1.27547641]])

In [44]:
from kddirkit.dataloaders import LoadFMA
feature_mode = "fma_all"
device = "cpu"
testDataLoader = LoadFMA.FMATestDataLoader(mode="pr", use_label = 'top' , feature_mode = feature_mode, device=device)


y_label_ = np.zeros((len(y_test_np), 16))
y_label_[np.arange(len(y_test_np)), y_test_np] = 1
y_label_

exclude_na_flatten_label = y_label_.reshape(-1)
exclude_na_label = y_label_
index_non_zero = np.sum(exclude_na_label, 0) > 0

evalMetric = metric.EvaFMATMetric(testDataLoader.id2genre,
                                               testDataLoader.fewrel_100,
                                               testDataLoader.fewrel_200,
                                               exclude_na_flatten_label,
                                               exclude_na_label,
                                               index_non_zero
                                               )

In [45]:
few_rel100 = ['Easy Listening', 'International', 'Blues', 'Spoken', 'Soul-RnB']
few_rel200 = ['Easy Listening', 'International', 'Blues', 'Spoken', 'Soul-RnB', 'Country']
labels = np.array(range(16))
labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [46]:
idx_few100_list = []
for i in range(len(X_test_np)):
    if y_test.iloc[i] in few_rel100:
        idx_few100_list.append(i)

idx_few200_list = []
for i in range(len(X_test_np)):
    if y_test.iloc[i] in few_rel200:
        idx_few200_list.append(i)
print(len(idx_few100_list) , len(idx_few200_list))

x_test_few_100 = X_test_np[idx_few100_list]
y_test_few_100 = y_test_np[idx_few100_list]
x_test_few_200 = X_test_np[idx_few200_list]
y_test_few_200 = y_test_np[idx_few200_list]

170 188


## MLP

In [47]:
mlp_model = MLPClassifier()
mlp_model.fit(X_train_np, y_train_np)
score = mlp_model.score(X_test_np, y_test_np)
print('Accuracy: {:.2%}'.format(score))

Accuracy: 41.55%


In [48]:
x_test_few_100_prob = mlp_model.predict_proba(x_test_few_100)
x_test_few_200_prob = mlp_model.predict_proba(x_test_few_200)

y_test_few_100_ = np.zeros((len(y_test_few_100), 16))
y_test_few_100_[np.arange(len(y_test_few_100)), y_test_few_100] = 1
y_test_few_100_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [49]:
top2score = top_k_accuracy_score(y_test_few_100, x_test_few_100_prob, k=2, labels= labels)
top2score

0.1

In [50]:
top2score = top_k_accuracy_score(y_test_few_100, x_test_few_100_prob, k=3, labels= labels)
top2score

0.13529411764705881

In [51]:
top2score = top_k_accuracy_score(y_test_few_100, x_test_few_100_prob, k=5, labels= labels)
top2score

0.3411764705882353

In [52]:
top2score = top_k_accuracy_score(y_test_few_200, x_test_few_200_prob, k=2, labels= labels)
top2score

0.09042553191489362

In [53]:
top2score = top_k_accuracy_score(y_test_few_200, x_test_few_200_prob, k=3, labels= labels)
top2score

0.12234042553191489

In [ ]:
top2score = top_k_accuracy_score(y_test_few_200, x_test_few_200_prob, k=5, labels= labels)
top2score

In [58]:
rand = torch.rand((30, 16))

In [64]:
F.softmax(rand, dim= 1).shape

torch.Size([30, 16])

In [ ]:
torch.rand((30, 16))

In [70]:
torch.sum(rand, dim = 0).shape

torch.Size([16])